# LangChain Expression Language (LCEL): Advanced Chain Composition

## Introduction

**LCEL (LangChain Expression Language)** is the modern way to compose LangChain components using declarative syntax. It makes chains readable, composable, and powerful.

### What is LCEL?

LCEL allows you to:
- **Compose chains** with the pipe operator `|`
- **Run in parallel** with `RunnableParallel`
- **Add conditional logic** with `RunnableBranch`
- **Create custom functions** with `RunnableLambda`
- **Handle errors gracefully** with fallbacks and retries
- **Stream results** automatically

### Why LCEL?

| ❌ Legacy Chains | ✅ LCEL |
|-----------------|--------|
| `LLMChain(llm=llm, prompt=prompt)` | `prompt \| model \| parser` |
| Hard to compose | Pipe operator intuitive |
| Limited streaming | Built-in streaming |
| Verbose syntax | Concise and readable |

### Core Concept: Runnables

Everything in LCEL is a `Runnable` with these methods:
- `.invoke(input)` - Synchronous execution
- `.stream(input)` - Streaming results
- `.batch(inputs)` - Batch processing
- `.ainvoke(input)` - Async execution

---

## Installation & Setup

In [ ]:
import os
from getpass import getpass

# Set API key
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API Key: ")

print("API key configured!")

---

## Example 1: Basic Pipe Operator

The pipe `|` operator chains components together:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Create components
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
model = ChatOpenAI(model="gpt-4")
parser = StrOutputParser()

# Chain with pipe operator (reads like a data flow!)
chain = prompt | model | parser

# Invoke
result = chain.invoke({"topic": "programming"})
print(result)

### What's Happening?

1. `prompt` receives `{"topic": "programming"}` → creates formatted message
2. `model` receives message → generates AIMessage
3. `parser` receives AIMessage → extracts string

The `|` operator automatically passes output from left to right!

---

## Example 2: RunnablePassthrough

Pass input through unchanged (useful for parallel operations):

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Simple passthrough example
passthrough = RunnablePassthrough()
result = passthrough.invoke({"key": "value"})
print(f"Passthrough result: {result}")

# Use in chain to preserve input
prompt = ChatPromptTemplate.from_template("Explain {concept} in simple terms")
chain = {"concept": RunnablePassthrough()} | prompt | ChatOpenAI(model="gpt-4") | StrOutputParser()

# Input is just a string, passthrough makes it dict with 'concept' key
result = chain.invoke("recursion")
print(f"\nExplanation:\n{result}")

---

## Example 3: RunnableParallel - Parallel Execution

Run multiple operations in parallel and combine results:

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI(model="gpt-4")

# Create multiple chains that run in parallel
joke_chain = (
    ChatPromptTemplate.from_template("Tell a joke about {topic}")
    | model
    | StrOutputParser()
)

fact_chain = (
    ChatPromptTemplate.from_template("Tell an interesting fact about {topic}")
    | model
    | StrOutputParser()
)

poem_chain = (
    ChatPromptTemplate.from_template("Write a haiku about {topic}")
    | model
    | StrOutputParser()
)

# Run all in parallel
parallel_chain = RunnableParallel(
    joke=joke_chain,
    fact=fact_chain,
    poem=poem_chain
)

# Execute (all three LLM calls happen in parallel!)
result = parallel_chain.invoke({"topic": "Python"})

print("Joke:")
print(result["joke"])
print("\nFact:")
print(result["fact"])
print("\nPoem:")
print(result["poem"])

### Performance Benefits

- **Sequential**: 3 chains × 2 seconds each = 6 seconds total
- **Parallel**: max(2, 2, 2) = ~2 seconds total

**3x faster!**

---

## Example 4: Dict Syntax for RunnableParallel

Shorthand syntax using dictionaries:

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI(model="gpt-4")

# Dict syntax automatically creates RunnableParallel
chain = (
    # Pass through input and add uppercase version
    {
        "original": RunnablePassthrough(),
        "uppercase": lambda x: x["word"].upper(),
        "length": lambda x: len(x["word"])
    }
    # These all run in parallel!
)

result = chain.invoke({"word": "python"})
print(result)

---

## Example 5: RunnableLambda - Custom Functions

Wrap Python functions as Runnables:

In [ ]:
from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Define custom functions
def preprocess_input(input_dict):
    """Clean and prepare input."""
    text = input_dict["text"]
    return {"cleaned_text": text.strip().lower()}

def postprocess_output(text):
    """Format output."""
    return f"[PROCESSED] {text.upper()}"

# Wrap as Runnables
preprocess = RunnableLambda(preprocess_input)
postprocess = RunnableLambda(postprocess_output)

# Build chain
prompt = ChatPromptTemplate.from_template("Explain: {cleaned_text}")
chain = preprocess | prompt | ChatOpenAI(model="gpt-4") | StrOutputParser() | postprocess

result = chain.invoke({"text": "  Python Decorators  "})
print(result)

### Shorthand: Direct Lambda

You can use lambda functions directly without wrapping:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Lambda directly in chain (auto-wrapped as RunnableLambda)
chain = (
    ChatPromptTemplate.from_template("{number}")
    | ChatOpenAI(model="gpt-4")
    | StrOutputParser()
    | (lambda x: f"Result: {x}")
)

result = chain.invoke({"number": "What is 2+2?"})
print(result)

---

## Example 6: RunnableBranch - Conditional Logic

Route to different chains based on conditions:

In [ ]:
from langchain_core.runnables import RunnableBranch
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI(model="gpt-4")

# Different chains for different languages
python_chain = (
    ChatPromptTemplate.from_template("Explain this Python concept: {topic}")
    | model
    | StrOutputParser()
)

javascript_chain = (
    ChatPromptTemplate.from_template("Explain this JavaScript concept: {topic}")
    | model
    | StrOutputParser()
)

general_chain = (
    ChatPromptTemplate.from_template("Explain this concept: {topic}")
    | model
    | StrOutputParser()
)

# Create branching logic
branch = RunnableBranch(
    # (condition, runnable) pairs
    (lambda x: x.get("language") == "python", python_chain),
    (lambda x: x.get("language") == "javascript", javascript_chain),
    # Default (no condition)
    general_chain
)

# Test different branches
print("Python branch:")
result = branch.invoke({"language": "python", "topic": "decorators"})
print(result[:100] + "...\n")

print("JavaScript branch:")
result = branch.invoke({"language": "javascript", "topic": "promises"})
print(result[:100] + "...\n")

print("Default branch:")
result = branch.invoke({"language": "unknown", "topic": "algorithms"})
print(result[:100] + "...")

---

## Example 7: Error Handling with Fallbacks

Gracefully handle failures by falling back to alternatives:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("Tell me about {topic}")
parser = StrOutputParser()

# Primary: Expensive model
primary_chain = prompt | ChatOpenAI(model="gpt-4", timeout=1) | parser

# Fallback 1: Cheaper model
fallback1_chain = prompt | ChatOpenAI(model="gpt-3.5-turbo") | parser

# Fallback 2: Simple response
fallback2_chain = lambda x: f"Sorry, I couldn't process your request about {x['topic']}"

# Chain with fallbacks
chain_with_fallbacks = primary_chain.with_fallbacks(
    [fallback1_chain, fallback2_chain]
)

# If GPT-4 fails (timeout/error), tries GPT-3.5, then fallback message
result = chain_with_fallbacks.invoke({"topic": "Python"})
print(result)

---

## Example 8: Retry Logic

Automatically retry on failures:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("What is {topic}?")
model = ChatOpenAI(model="gpt-4")
parser = StrOutputParser()

chain = prompt | model | parser

# Add retry logic (retries up to 3 times on failure)
chain_with_retry = chain.with_retry(
    stop_after_attempt=3,
    wait_exponential_jitter=True  # Exponential backoff with jitter
)

result = chain_with_retry.invoke({"topic": "async programming"})
print(result)

---

## Example 9: Streaming with LCEL

LCEL chains support streaming by default:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

chain = (
    ChatPromptTemplate.from_template("Write a short story about {topic}")
    | ChatOpenAI(model="gpt-4")
    | StrOutputParser()
)

# Stream tokens as they arrive
print("Streaming story:\n")
for chunk in chain.stream({"topic": "a robot learning to code"}):
    print(chunk, end="", flush=True)

print("\n\nDone!")

---

## Example 10: Batch Processing

Process multiple inputs efficiently:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

chain = (
    ChatPromptTemplate.from_template("What is the capital of {country}?")
    | ChatOpenAI(model="gpt-4", temperature=0)
    | StrOutputParser()
)

# Batch inputs
countries = [
    {"country": "France"},
    {"country": "Japan"},
    {"country": "Brazil"},
    {"country": "Egypt"}
]

# Process in batch (more efficient than individual calls)
results = chain.batch(countries)

for country, capital in zip(countries, results):
    print(f"{country['country']}: {capital}")

---

## Example 11: Async Execution

Use async methods for concurrent processing:

In [ ]:
import asyncio
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

chain = (
    ChatPromptTemplate.from_template("Explain {concept} in one sentence")
    | ChatOpenAI(model="gpt-4")
    | StrOutputParser()
)

async def process_concepts():
    concepts = ["recursion", "polymorphism", "encapsulation"]
    
    # Process concurrently
    tasks = [chain.ainvoke({"concept": c}) for c in concepts]
    results = await asyncio.gather(*tasks)
    
    for concept, result in zip(concepts, results):
        print(f"{concept}: {result}\n")

# Run in Jupyter
await process_concepts()

---

## Example 12: Complex Chain - RAG-like Pattern

Combine multiple LCEL features:

In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Simulate document retrieval
def retrieve_context(query):
    """Simulate fetching relevant context."""
    # In real RAG, this would query a vector store
    contexts = {
        "What are design patterns?": "Design patterns are reusable solutions to common software design problems.",
        "What is Python?": "Python is a high-level, interpreted programming language known for readability."
    }
    return contexts.get(query["question"], "No context found.")

# Build RAG-like chain
prompt = ChatPromptTemplate.from_template(
    "Context: {context}\n\nQuestion: {question}\n\nAnswer:"
)

chain = (
    # Step 1: Get question and retrieve context in parallel
    RunnableParallel(
        context=retrieve_context,
        question=RunnablePassthrough()
    )
    # Step 2: Format prompt with context + question
    | (lambda x: {"context": x["context"], "question": x["question"]["question"]})
    # Step 3: Get answer from LLM
    | prompt
    | ChatOpenAI(model="gpt-4")
    | StrOutputParser()
)

result = chain.invoke({"question": "What are design patterns?"})
print(result)

---

## Example 13: itemgetter for Extracting Keys

Use `itemgetter` to extract specific keys:

In [ ]:
from operator import itemgetter
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Chain that uses only specific keys
prompt = ChatPromptTemplate.from_template(
    "Translate '{text}' to {language}"
)

chain = (
    # Extract only needed keys from input dict
    {
        "text": itemgetter("text"),
        "language": itemgetter("target_language")
    }
    | prompt
    | ChatOpenAI(model="gpt-4")
    | StrOutputParser()
)

# Input has extra keys that are ignored
result = chain.invoke({
    "text": "Hello",
    "target_language": "Spanish",
    "extra_key": "ignored"
})

print(result)

---

## Example 14: Multi-Step Chain with State

Build chains that pass state between steps:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

model = ChatOpenAI(model="gpt-4")

# Step 1: Generate outline
outline_chain = (
    ChatPromptTemplate.from_template("Create a 3-point outline for: {topic}")
    | model
    | StrOutputParser()
)

# Step 2: Expand outline
expand_chain = (
    ChatPromptTemplate.from_template(
        "Topic: {topic}\nOutline: {outline}\n\nExpand this outline into a full explanation:"
    )
    | model
    | StrOutputParser()
)

# Combine steps while preserving topic
full_chain = (
    {"topic": RunnablePassthrough()}
    | RunnablePassthrough.assign(outline=outline_chain)
    | expand_chain
)

result = full_chain.invoke({"topic": "Python asyncio"})
print(result)

---

## Real-World Example: Code Review Chain

Complete example combining many LCEL features:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

model = ChatOpenAI(model="gpt-4")

# Parallel analysis chains
bugs_prompt = ChatPromptTemplate.from_template(
    "Find potential bugs in this code. Return JSON with 'bugs' list:\n{code}"
)
bugs_chain = bugs_prompt | model | JsonOutputParser()

style_prompt = ChatPromptTemplate.from_template(
    "Analyze code style. Return JSON with 'issues' list:\n{code}"
)
style_chain = style_prompt | model | JsonOutputParser()

performance_prompt = ChatPromptTemplate.from_template(
    "Analyze performance. Return JSON with 'suggestions' list:\n{code}"
)
performance_chain = performance_prompt | model | JsonOutputParser()

# Run all analyses in parallel
analysis_chain = RunnableParallel(
    code=RunnablePassthrough(),
    bugs=bugs_chain,
    style=style_chain,
    performance=performance_chain
)

# Synthesis chain
synthesis_prompt = ChatPromptTemplate.from_template(
    "Code:\n{code}\n\nBugs: {bugs}\nStyle: {style}\nPerformance: {performance}\n\n"
    "Provide a summary code review:"
)
synthesis_chain = synthesis_prompt | model | StrOutputParser()

# Full chain
code_review_chain = analysis_chain | synthesis_chain

# Test it
code_sample = """
def calculate_sum(numbers):
    total = 0
    for i in range(len(numbers)):
        total = total + numbers[i]
    return total
"""

result = code_review_chain.invoke({"code": code_sample})
print(result)

---

## Best Practices

### ✅ Do

1. **Use pipe operator** for sequential operations
2. **Use RunnableParallel** for independent operations
3. **Use RunnableBranch** for conditional logic
4. **Add fallbacks** for production reliability
5. **Use RunnablePassthrough** to preserve input
6. **Leverage streaming** for better UX
7. **Use async** for I/O-bound operations

### ❌ Don't

1. **Don't use legacy chains** (LLMChain, etc.)
2. **Don't run independent operations sequentially** (use parallel)
3. **Don't ignore errors** (add fallbacks/retries)
4. **Don't make chains too complex** (break into smaller pieces)
5. **Don't forget type hints** (helps with debugging)

---

## Common Pitfalls

### ❌ Mistake 1: Not Preserving Input

```python
# Wrong - loses original input
chain = transform | prompt | model
```

**Solution**: Use RunnablePassthrough to preserve:
```python
chain = {"original": RunnablePassthrough(), "transformed": transform} | ...
```

### ❌ Mistake 2: Sequential Instead of Parallel

```python
# Slow - runs sequentially
result1 = chain1.invoke(input)
result2 = chain2.invoke(input)
```

**Solution**: Use RunnableParallel:
```python
parallel = RunnableParallel(r1=chain1, r2=chain2)
results = parallel.invoke(input)
```

### ❌ Mistake 3: No Error Handling

```python
# Fragile - fails completely on any error
chain = prompt | expensive_model | parser
```

**Solution**: Add fallbacks:
```python
chain = (prompt | expensive_model | parser).with_fallbacks([cheap_model_chain])
```

---

## Practice Exercises

In [ ]:
# Exercise 1: Create a research chain
# 1. Generate 3 questions about a topic
# 2. Answer each question in parallel
# 3. Synthesize answers into summary

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.runnables import RunnablePassthrough

# Your code here:
model = ChatOpenAI(model="gpt-4")

# Step 1: Generate questions
questions_chain = (
    ChatPromptTemplate.from_template(
        "Generate 3 questions about {topic}. Return JSON with 'questions' array."
    )
    | model
    | JsonOutputParser()
)

# Step 2: Answer questions (you'll need to implement this)
# Step 3: Synthesize (you'll need to implement this)

# Test your chain
# result = research_chain.invoke({"topic": "Python decorators"})
# print(result)

In [ ]:
# Exercise 2: Create a sentiment analysis chain with fallback
# - Try using expensive model first
# - Fall back to simple keyword matching if it fails

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Your code here:
def keyword_sentiment(input_dict):
    """Simple keyword-based sentiment."""
    text = input_dict["text"].lower()
    if any(word in text for word in ["love", "great", "amazing"]):
        return "positive"
    elif any(word in text for word in ["hate", "terrible", "awful"]):
        return "negative"
    return "neutral"

# Create chain with fallback
# ...

In [ ]:
# Exercise 3: Create a multi-language translation chain
# - Detect source language
# - Translate to target language
# - Run in parallel: translate to Spanish, French, German

from langchain_core.runnables import RunnableParallel
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Your code here:
model = ChatOpenAI(model="gpt-4")

# Create chains for each language
# Combine with RunnableParallel
# ...

---

## Key Takeaways

### ✅ What We Learned

1. **Pipe Operator `|`**: Chain components sequentially
2. **RunnablePassthrough**: Preserve input through chain
3. **RunnableParallel**: Execute independent operations concurrently
4. **RunnableBranch**: Conditional routing to different chains
5. **RunnableLambda**: Wrap Python functions as Runnables
6. **Error Handling**: Fallbacks and retries for reliability
7. **Streaming**: Built-in support for token-by-token output
8. **Batch/Async**: Efficient processing of multiple inputs

### 📚 Next Steps

- **langchain_rag.ipynb**: Apply LCEL to RAG pipelines
- **langchain_agents.ipynb**: Use LCEL with agents
- **langchain_memory.ipynb**: Add memory to LCEL chains

---

## Resources

- [LCEL Documentation](https://python.langchain.com/docs/expression_language/)
- [Runnable Interface](https://python.langchain.com/docs/expression_language/interface)
- [LCEL Cookbook](https://python.langchain.com/docs/expression_language/cookbook)
- [Streaming Guide](https://python.langchain.com/docs/expression_language/streaming)

---

**Next Notebook**: `langchain_rag.ipynb` - Build complete RAG applications with LCEL